# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [1]:
import matplotlib.pyplot as plt
import mplleaflet
import matplotlib.ticker as ticker
import pandas as pd
import numpy as np
import calendar

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

In [2]:
Data= pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
Data['Data_Value']=0.1*Data ['Data_Value']
Data = Data[~(Data['Date'].astype('str').str.contains('-02-29',regex=False))]

Data['Date']=pd.to_datetime(Data['Date'])

Data_2015= Data[Data['Date']>='2015-01-01'].copy()
NData=Data[Data['Date']<'2015-01-01'].copy()

Data=Data.groupby('Date').agg({'Data_Value': {'larger':'max', 'smaller':'min'}})

#print(Data)

In [3]:
NData['Date']=list(map(lambda dt: dt.replace(year=2001),NData['Date']))

T_max= NData.groupby('Date').agg({'Data_Value': 'max'})
T_min= NData.groupby('Date').agg({'Data_Value':'min'})

Max_2015= Data_2015.groupby('Date').agg({'Data_Value': 'max'})
Min_2015= Data_2015.groupby('Date').agg({'Data_Value': 'min'})

T_max['Data_2015']=Max_2015['Data_Value'].tolist()
T_min['Data_2015']=Min_2015['Data_Value'].tolist()

In [4]:
T_min.index=np.arange(365)
T_max.index=np.arange(365)

y_min= Data[('Data_Value','smaller')].min()
x_min= Data[Data[('Data_Value','smaller')]==y_min].index.strftime('%d %b-%Y')

y_max= Data[('Data_Value','larger')].max()
x_max= Data[Data[('Data_Value','larger')]==y_max].index.strftime('%d %b-%Y')

In [5]:
Test_max=T_max['Data_2015']>T_max['Data_Value']
Test_min=T_min['Data_2015']<T_min['Data_Value']
x_val= pd.date_range('2001-01-01','2001-12-31', freq='D').strftime('%d %b')

In [67]:
plt.close()
plt.figure(figsize=(40,20))
axislabel_size=20

plt.plot(T_min.index,T_min['Data_Value'].tolist(),':',color='black', alpha=0.5, label='Record low (2000-2014)')
plt.plot(T_min.index,T_max['Data_Value'].tolist(),'-',color='black', alpha=0.5, label='Record high (2000-2014)')

AX=plt.gca()
                       
AX.fill_between(T_min.index,T_min['Data_Value'].tolist(),T_max['Data_Value'].tolist(),facecolor='black',alpha=0.05)

plt.xticks(T_min.index[0::7], x_val[0::7], rotation=90)
plt.tick_params(axis='both', which='major', labelsize=axislabel_size,pad=10)

plt.scatter(T_min[Test_min].index,T_min[Test_min].Data_2015,s=50,marker="x", color='blue',label='Record breaking highs in 2015')
plt.scatter(T_max[Test_max].index,T_max[Test_max].Data_2015,s=100,marker=".", color='red',label='Record breaking lows in 2015')

#plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %y'))
#plt.gca().xaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: x_val.strftime('%m-%d')))

In [68]:
plt.title('Ann Harbour Daily Record Temperatures for 2000-2015', fontsize=28)
AX.set_xlabel('Days of the Year (W/O 29 Feb)', fontsize=axislabel_size, labelpad=20)
AX.set_ylabel ('Temperature [$\degree$C]', fontsize=axislabel_size, labelpad=10)

AX.spines['right'].set_visible(False)
AX.spines['top'].set_visible(False)
AX.spines['left'].set_position(('outward', 10))

plt.axis([0, 365, -40, 50])

# Zero C reference temperature
AX.axhline(y=0, xmin=0, xmax=365,linestyle=(0,(5,10)), color='black', alpha=0.5) # zero temp line
#AX.axvline(x=np.where(x_val=='01 Sep'), ymin=-40, ymax=40, color='black', alpha=0.5 ) #to do seasons

minlabel='Global Min \n( {} , {} $\degree$C )'.format(x_min[0],round(y_min,2))
maxlabel='Global Max'

xmin_index=x_val.tolist().index(x_min[0][:-5])
xmax_index=[x_val.tolist().index(x[:-5]) for x in x_max]

#Global min annotation
plt.annotate(minlabel, xy=(xmin_index,y_min), xytext=(xmin_index+10, y_min+2),
             arrowprops=dict(facecolor='black',headlength=7, headwidth=5,width=0.6),
             va = "top", ha="left",fontsize=axislabel_size)

#Global max annotation (multiple instances)
for i in range(len(xmax_index)):  
    maxlabel+='\n( {} , {} $\degree$C ) '.format(x_max[i],round(y_max,2))
    plt.annotate('', xy=(xmax_index[i],y_max), xytext=(xmax_index[0]+10, y_max+5),
                 arrowprops=dict(facecolor='black',headlength=7, headwidth=5,width=0.6))

plt.annotate(' '+maxlabel, xy=(xmax_index[0],y_max), xytext=(xmax_index[0]+10, y_max+7),
             va = "top", ha="left", fontsize=axislabel_size)

#T=0 horizontal line annotation
plt.annotate('T= 0 $\degree$C', xy=(x_val.size/2,0), xytext=(x_val.size/2, -2), fontsize=axislabel_size)

plt.legend(fontsize=axislabel_size, loc=1)
plt.savefig('Ann_Harbour_Daily_Temp.pdf',bbox_inches='tight')

In [19]:
plt.show()